# Mask R-CNN for Bin Picking

This notebook is adopted from the [TorchVision 0.3 Object Detection finetuning tutorial](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html).  We will be finetuning a pre-trained [Mask R-CNN](https://arxiv.org/abs/1703.06870) model on a dataset generated from our "clutter generator" script.


In [ ]:
!pip3 install --update pycocotools


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --update


In [ ]:
!pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# Download TorchVision repo to use some files from
# references/detection
!git clone https://github.com/pytorch/vision.git
!cd vision && git checkout v0.3.0
!cp vision/references/detection/utils.py ./
!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./


  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-b4f15wdf
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-b4f15wdf
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0-cp310-cp310-linux_x86_64.whl size=376057 sha256=cd9992ff03f56f2f5e3b0dbc5975d57b34d0ff85bf3a3f5d762dcd87dadfdb89
  Stored in directory: /tmp/pip-ephem-wheel-cache-8f5_mwk1/wheels/39/61/b4/480fbddb4d3d6bc34083e7397bc6f5d1381f79acc68e9f3511
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.8
    Uninstalling pycocotools-2.0.8:
      Successfully uninstalled pycocotools-2.0.8
Cloning into 'vision'...
remote: Enumerating objects: 586801, done.
remote: Counting objects: 100% (34587/34587), done.
remote: Compressing obje

In [ ]:
# Imports
import fnmatch
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.utils.data
from PIL import Image

ycb = [
    "003_cracker_box.sdf",
    "004_sugar_box.sdf",
    "005_tomato_soup_can.sdf",
    "006_mustard_bottle.sdf",
    "009_gelatin_box.sdf",
    "010_potted_meat_can.sdf",
    "011_cup.sdf",
    "012_ball.sdf",
]

# drake_reserved_labels = [32765, 32764, 32766, 32767]


def colorize_labels(image):
    """Colorizes labels."""
    cc = mpl.colors.ColorConverter()
    color_cycle = plt.rcParams["axes.prop_cycle"]
    colors = np.array([cc.to_rgb(c["color"]) for c in color_cycle])
    bg_color = [0, 0, 0]
    image = np.squeeze(image)
    background = np.zeros(image.shape[:2], dtype=bool)
    for label in reserved_labels:
        background |= image == int(label)
    image[np.logical_not(background)]
    color_image = colors[image % len(colors)]
    color_image[background] = bg_color
    return color_image

# Download our bin-picking dataset

It's definitely possible to actually create this dataset on Colab; I've just written a version of the "clutter_gen" method from the last chapter that writes the images (and label images) to disk, along with some annotations.  But it takes a non-trivial amount of time to generate 10,000 images.


In [ ]:
# dataset_path = "clutter_maskrcnn_data"
# if not os.path.exists(dataset_path):
#     !wget https://groups.csail.mit.edu/locomotion/clutter_maskrcnn_data.zip .
!unzip -q clutter_maskrcnn_data.zip

If you are on colab, go ahead and use the file browser on the left (looks like a drive under the table of contents panel) to click through the .png and .json files to make sure you understand the dataset you've just created!  If you're on a local machine, just browse to the folder.

# Teach pytorch how to load the dataset

into the [format expected by Mask R-CNN](https://pytorch.org/docs/stable/torchvision/models.html#torchvision.models.detection.maskrcnn_resnet50_fpn).

In [ ]:
class BinPickingDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.num_images = len(fnmatch.filter(os.listdir(root), "*.png"))
        self.transforms = transforms

    def __getitem__(self, idx):
        filename_base = os.path.join(self.root, f"{idx:05d}")

        img = Image.open(filename_base + ".png").convert("RGB")
        mask = np.squeeze(np.load(filename_base + "_mask.npy"))

        if idx > 9999:
          mask[mask == 1] = 2

        with open(filename_base + ".json", "r") as f:
            instance_id_to_class_name = json.load(f)
        labels = ycb == instance_id_to_class_name

        # instances are encoded as different colors
        obj_ids = np.asarray(list(instance_id_to_class_name.keys()))
        count = (mask == np.int16(obj_ids)[:, None, None]).sum(axis=2).sum(axis=1)

        # discard objects instances with less than 10 pixels
        obj_ids = obj_ids[count >= 10]

        labels = [ycb.index(instance_id_to_class_name[id] + ".sdf") for id in obj_ids]
        obj_ids = np.int16(np.asarray(obj_ids))

        # split the color-encoded mask into a set of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return self.num_images

Let's check the output of our dataset.

In [ ]:
dataset_path = "clutter_maskrcnn_data"
np.set_printoptions(threshold=np.inf)

In [ ]:
plt.imshow(np.squeeze(np.load(os.path.join(dataset_path, f"09999_mask.npy"))))
print(np.load(os.path.join(dataset_path, f"09999_mask.npy")))

In [ ]:
dataset = BinPickingDataset(dataset_path)
dataset[10000][1]['masks'].shape

torch.Size([1, 480, 640])

# Define the network

This cell is where the magic begins to happen.  We load a network that is pre-trained on the COCO dataset, then replace the network head with a new (untrained) network with the right number of outputs for our YCB recognition/segmentation task.

In [ ]:
import torchvision
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(
        weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT
    )

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask, hidden_layer, num_classes
    )

    return model

That's it, this will make model be ready to be trained and evaluated on our custom dataset.

# Transforms

Let's write some helper functions for data augmentation / transformation, which leverages the functions in torchvision `refereces/detection`.


In [ ]:
import transforms as T
import utils
from engine import evaluate, train_one_epoch

def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

Note that we do not need to add a mean/std normalization nor image rescaling in the data transforms, as those are handled internally by the Mask R-CNN model.

# Putting everything together

We now have the dataset class, the models and the data transforms. Let's instantiate them

In [ ]:
# use our dataset and defined transformations
dataset = BinPickingDataset(dataset_path, get_transform(train=True))
dataset_test = BinPickingDataset(dataset_path, get_transform(train=False))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    collate_fn=utils.collate_fn,
)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=utils.collate_fn,
)

Now let's instantiate the model and the optimizer

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

num_classes = len(ycb) + 1

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:00<00:00, 190MB/s]


And now let's train the model for 10 epochs, evaluating at the end of every epoch.

In [ ]:
# let's train it for 10 epochs
num_epochs = 10

for epoch in range(num_epochs):
    # evaluate(model, data_loader_test, device=device)
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=500)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

Epoch: [0]  [   0/5175]  eta: 0:25:22  lr: 0.000010  loss: 1.2122 (1.2122)  loss_classifier: 0.2846 (0.2846)  loss_box_reg: 0.3542 (0.3542)  loss_mask: 0.1343 (0.1343)  loss_objectness: 0.4087 (0.4087)  loss_rpn_box_reg: 0.0304 (0.0304)  time: 0.2941  data: 0.1553  max mem: 3826
Epoch: [0]  [ 500/5175]  eta: 0:08:05  lr: 0.002507  loss: 0.3481 (0.4639)  loss_classifier: 0.0749 (0.1337)  loss_box_reg: 0.1272 (0.1903)  loss_mask: 0.0875 (0.1179)  loss_objectness: 0.0008 (0.0123)  loss_rpn_box_reg: 0.0032 (0.0096)  time: 0.1023  data: 0.0072  max mem: 4106
Epoch: [0]  [1000/5175]  eta: 0:07:11  lr: 0.005000  loss: 0.2143 (0.3736)  loss_classifier: 0.0425 (0.1019)  loss_box_reg: 0.1016 (0.1539)  loss_mask: 0.0662 (0.0998)  loss_objectness: 0.0021 (0.0098)  loss_rpn_box_reg: 0.0041 (0.0082)  time: 0.1027  data: 0.0076  max mem: 4106
Epoch: [0]  [1500/5175]  eta: 0:06:19  lr: 0.005000  loss: 0.2462 (0.3306)  loss_classifier: 0.0523 (0.0866)  loss_box_reg: 0.1031 (0.1358)  loss_mask: 0.0581 (

If you're going to leave this running for a bit, I recommend scheduling the following cell to run immediately (so that you don't lose your work).

In [ ]:
torch.save(model.state_dict(), "clutter_maskrcnn_model.pt")

from google.colab import files

files.download("clutter_maskrcnn_model.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Now that training has finished, let's have a look at what it actually predicts in a test image

In [ ]:
# pick one image from the test set
img, _ = dataset_test[0]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])

Printing the prediction shows that we have a list of dictionaries. Each element of the list corresponds to a different image. As we have a single image, there is a single dictionary in the list.
The dictionary contains the predictions for the image we passed. In this case, we can see that it contains `boxes`, `labels`, `masks` and `scores` as fields.

In [ ]:
prediction

Let's inspect the image and the predicted segmentation masks.

For that, we need to convert the image, which has been rescaled to 0-1 and had the channels flipped so that we have it in `[C, H, W]` format.

In [ ]:
Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())

And let's now visualize the top predicted segmentation mask. The masks are predicted as `[N, 1, H, W]`, where `N` is the number of predictions, and are probability maps between 0-1.

In [ ]:
Image.fromarray(prediction[0]["masks"][0, 0].mul(255).byte().cpu().numpy())

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=22327430-09f7-4736-a7ab-08636417e56c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>